Dataanalyse og Machine Learning (F24.520214U002.A)  
Gruppe 1:  
Hans Christian Hulvej - AU19930532  
Kathrine Bohus Madsen - AU202301227  
Tor Kolding - AU20000148  
Torben Holst Rendboe – AU20064217    
  
Opgave 3  
  
Deep Learning classification tensorflow med tensorboard  

Iterativ gennemløb af: *epochs*, *neurons*, *layers*, *dropout* & *Optimazion type* holdt op mod *accuracy* med henblik på at finde den **optimale kombination**.

D. 26-04-2024

In [367]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import datetime
import itertools
import os
import shutil

# Load the TensorBoard notebook extension.
%reload_ext tensorboard

Vi starter med at slette tidligere tensorboard logs.  
Hvis man vil vise alle logs, udkommenteres / slettes disse to liner kode.

In [368]:
shutil.rmtree('logs/fit') 
os.mkdir('logs/fit')

Vi starter med at hente datasettet, i dette tilfælde er det **WineQT.csv**  
Så fjerne vi Id kolonnen, da den ikke tjener noget formål og skriver lige info ud for at tjekke at det er som vi ønsker.  
Så deler vi datasettet op i *features* og *target*. 

In [369]:
data = pd.read_csv('WineQT.csv')
data = data.drop(columns=['Id'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1143 non-null   float64
 1   volatile acidity      1143 non-null   float64
 2   citric acid           1143 non-null   float64
 3   residual sugar        1143 non-null   float64
 4   chlorides             1143 non-null   float64
 5   free sulfur dioxide   1143 non-null   float64
 6   total sulfur dioxide  1143 non-null   float64
 7   density               1143 non-null   float64
 8   pH                    1143 non-null   float64
 9   sulphates             1143 non-null   float64
 10  alcohol               1143 non-null   float64
 11  quality               1143 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 107.3 KB


In [370]:
features = data.drop(columns=["quality"])
target = data["quality"]

Vi anvender **StandarScaler** klassen til at normaliserer vores *features*, og gemmer resultatet i *X_scaled*.  
Vi gemmer også target i *y*  
  
Så deler vi vores data (*X_scaled* og *y*) op i train-set og test-set, med en fordeling på 80% til træning og 20% til testning.  
Vi sætter og *random_state* til 42 (fast seed).

In [371]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)
y = target

In [372]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

<div class="alert-warning" style="border-style: solid; padding:5px;">
<h2>FAKTA BOKS</h2>  
    
I denne model, arbejder vi med keras sequential model, som i princippet er en stak af lag, men især kendetegnet ved at hvert lag har en input- og en outputtensor. I dette tilfælde er der dog indtil videre kun et lag.  
  
Det er her i modellen at vi definerer hvormange fuldt forbundene neuroner der skal være (inden dropout).  
  
Derefter vælger vi hvor stort vores dropout skal være.
</div>

Vi bygger vores sequential model med et lag hvoi der er 128 neuroner, med **relu** som aktiveringstype og et dropout på 20 %

In [373]:
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(11,)),
    layers.Dropout(0.2),
    layers.Dense(11)  # Output layer with 11 units (one for each quality score)
])

<div class="alert-warning" style="border-style: solid; padding:5px;">
<h2>FAKTA BOKS</h2>  
  
Så skal vi have modellen konfigureret til træning, det gør vi ved at *compilerer* den.  
  
Når vi compilerer, skal vi vælge hvilken optimeringsalgoritme vi ønsker at bruge, i dette tilfælde er det **adam**, som bestemmer måden hvorpå modellen evaluerer.  
  
Og tilsidt konfigurer vi de/den data vi ønsker at overvåge under testen.  

</div>

Vi compilerer modellen med **adam** som optmeringsmetode og accuracy som den parameter vi vil se på.

In [374]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

<div class="alert-warning" style="border-style: solid; padding:5px;">
<h2>FAKTA BOKS</h2> 

Nu skal modellen trænes, med **model.fit**.  
  
Vi konfigurer *træningsdata* og *valideringsdata*.
  
Her konfigurer vi også hvormange gange vores træningssæt bliver "kørt" gennem modellen, *epochs*
    
Sluttelig sørger vi for at data bliver gjort tilgængelig for tensorboard, ved hjælp at *callbacks*.  
  
</div>

Så træner vi modellen med vores trænings data og valideringsdata, med en gennemkørsel på 20 gange.  
  
Inden da konfigurer vi stien til den placering hvor tensorboard skal hente sine logfiler og konfigurer vores callbacks (tensorboards og earlystop).

In [375]:
logdir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir) # eller "squared error" ved regression
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[tensorboard_callback, early_stopping_callback]) # 

Epoch 1/20
29/29 [==============================] - 1s 12ms/step - loss: 2.0940 - accuracy: 0.2976 - val_loss: 1.6536 - val_accuracy: 0.5415
Epoch 2/20
29/29 [==============================] - 0s 5ms/step - loss: 1.5013 - accuracy: 0.5481 - val_loss: 1.2519 - val_accuracy: 0.5764
Epoch 3/20
29/29 [==============================] - 0s 6ms/step - loss: 1.2426 - accuracy: 0.5427 - val_loss: 1.0808 - val_accuracy: 0.5808
Epoch 4/20
29/29 [==============================] - 0s 6ms/step - loss: 1.1382 - accuracy: 0.5667 - val_loss: 1.0065 - val_accuracy: 0.6026
Epoch 5/20
29/29 [==============================] - 0s 5ms/step - loss: 1.0714 - accuracy: 0.5799 - val_loss: 0.9679 - val_accuracy: 0.6201
Epoch 6/20
29/29 [==============================] - 0s 5ms/step - loss: 1.0412 - accuracy: 0.5919 - val_loss: 0.9425 - val_accuracy: 0.6245
Epoch 7/20
29/29 [==============================] - 0s 6ms/step - loss: 1.0287 - accuracy: 0.5853 - val_loss: 0.9268 - val_accuracy: 0.6201
Epoch 8/20
29/29 [=

Vi starter tensorboard  
  
Note: Hvis tensorboard ikke vil starte kan det skyldes at der ligger en PID fil og blokkere. Den kan fjernes ved at gå ind i  
c:\\bruger\<dit brugerid>\AppData\AppData\Local\Temp\.tensorboard-info og slette de PID filer der ligger der.

In [376]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 23980), started 23:26:19 ago. (Use '!kill 23980' to kill it.)

In [377]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", "%.2f" % (test_acc*100), " %")

8/8 [==============================] - 0s 3ms/step - loss: 0.8818 - accuracy: 0.6201
Test accuracy: 62.01  %


### Gennemløb med ændrede parameter ###
  
Så laver vi en optimeringstest, hvor vi iterer over epochs, neuron, dropout og optimerings metode.

In [378]:
no_epochs = [5, 10, 15, 20]
no_neurons = [64, 128, 256]
dropouts = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
optim_types = ['adam', 'sgd', 'rmsprop']
combs = list(itertools.product(no_epochs, no_neurons, dropouts, optim_types))

In [379]:
test_results = pd.DataFrame(columns=['epochs', 'neurons', 'dropout', 'Optimazion type', 'accuracy'])
test_results = test_results.head(0)

In [380]:
start_time = datetime.datetime.now()

In [381]:
for no_epochs, no_neurons, dropouts, optim_types in combs:
    model = tf.keras.Sequential([
        layers.Dense(no_neurons, activation='relu', input_shape=(11,)),
        layers.Dense(no_neurons, activation='relu'),
        layers.Dropout(dropouts),
        layers.Dense(11)  # Output layer with 11 units (one for each quality score)
    ])
    model.compile(optimizer=optim_types,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=no_epochs, validation_data=(X_test, y_test))
    acc = model.evaluate(X_test, y_test)[1]
    params = {'epochs':no_epochs, 'neurons':no_neurons, 'dropout': dropouts, 'Optimazion type': optim_types, 'accuracy': acc}
    test_results.loc[len(test_results)] = params

Epoch 1/5
29/29 [==============================] - 2s 16ms/step - loss: 1.8377 - accuracy: 0.4114 - val_loss: 1.3034 - val_accuracy: 0.5808
Epoch 2/5
29/29 [==============================] - 0s 5ms/step - loss: 1.2435 - accuracy: 0.5492 - val_loss: 1.0193 - val_accuracy: 0.5939
Epoch 3/5
29/29 [==============================] - 0s 5ms/step - loss: 1.0842 - accuracy: 0.5755 - val_loss: 0.9510 - val_accuracy: 0.6419
Epoch 4/5
29/29 [==============================] - 0s 5ms/step - loss: 1.0247 - accuracy: 0.5952 - val_loss: 0.9223 - val_accuracy: 0.6463
Epoch 5/5
8/8 [==============================] - 0s 3ms/step - loss: 0.9008 - accuracy: 0.6332
Epoch 1/5
29/29 [==============================] - 1s 14ms/step - loss: 2.3128 - accuracy: 0.1937 - val_loss: 2.1847 - val_accuracy: 0.3362
Epoch 2/5
29/29 [==============================] - 0s 5ms/step - loss: 2.0625 - accuracy: 0.4354 - val_loss: 1.9372 - val_accuracy: 0.4847
Epoch 3/5
29/29 [==============================] - 0s 5ms/step - loss

Epoch 4/5
29/29 [==============================] - 0s 6ms/step - loss: 1.5393 - accuracy: 0.4912 - val_loss: 1.3638 - val_accuracy: 0.5109
Epoch 5/5
8/8 [==============================] - 0s 3ms/step - loss: 1.2674 - accuracy: 0.5371
Epoch 1/5
29/29 [==============================] - 2s 18ms/step - loss: 1.7189 - accuracy: 0.4497 - val_loss: 1.2525 - val_accuracy: 0.5808
Epoch 2/5
29/29 [==============================] - 0s 10ms/step - loss: 1.2613 - accuracy: 0.5602 - val_loss: 1.0441 - val_accuracy: 0.6114
Epoch 3/5
29/29 [==============================] - 0s 7ms/step - loss: 1.1649 - accuracy: 0.5635 - val_loss: 0.9799 - val_accuracy: 0.6157
Epoch 4/5
29/29 [==============================] - 0s 5ms/step - loss: 1.0823 - accuracy: 0.5919 - val_loss: 0.9416 - val_accuracy: 0.6419
Epoch 5/5
8/8 [==============================] - 0s 4ms/step - loss: 0.9234 - accuracy: 0.6419
Epoch 1/5
29/29 [==============================] - 1s 12ms/step - loss: 1.8604 - accuracy: 0.3479 - val_loss: 1.3

29/29 [==============================] - 1s 14ms/step - loss: 1.6402 - accuracy: 0.4759 - val_loss: 1.1243 - val_accuracy: 0.5764
Epoch 2/5
29/29 [==============================] - 0s 5ms/step - loss: 1.1055 - accuracy: 0.5711 - val_loss: 0.9693 - val_accuracy: 0.6332
Epoch 3/5
29/29 [==============================] - 0s 5ms/step - loss: 1.0272 - accuracy: 0.5941 - val_loss: 0.9365 - val_accuracy: 0.6157
Epoch 4/5
29/29 [==============================] - 0s 5ms/step - loss: 0.9783 - accuracy: 0.6083 - val_loss: 0.9043 - val_accuracy: 0.6419
Epoch 5/5
8/8 [==============================] - 0s 4ms/step - loss: 0.8927 - accuracy: 0.6332
Epoch 1/5
29/29 [==============================] - 1s 18ms/step - loss: 2.2492 - accuracy: 0.2768 - val_loss: 2.0536 - val_accuracy: 0.5284
Epoch 2/5
29/29 [==============================] - 0s 7ms/step - loss: 1.9145 - accuracy: 0.4934 - val_loss: 1.7407 - val_accuracy: 0.5328
Epoch 3/5
29/29 [==============================] - 0s 6ms/step - loss: 1.6565 -

Epoch 4/5
29/29 [==============================] - 0s 4ms/step - loss: 1.4634 - accuracy: 0.5284 - val_loss: 1.3081 - val_accuracy: 0.5590
Epoch 5/5
8/8 [==============================] - 0s 2ms/step - loss: 1.2174 - accuracy: 0.5721
Epoch 1/5
29/29 [==============================] - 1s 12ms/step - loss: 1.5916 - accuracy: 0.4902 - val_loss: 1.1114 - val_accuracy: 0.5721
Epoch 2/5
29/29 [==============================] - 0s 4ms/step - loss: 1.1468 - accuracy: 0.5503 - val_loss: 0.9900 - val_accuracy: 0.5983
Epoch 3/5
29/29 [==============================] - 0s 4ms/step - loss: 1.0450 - accuracy: 0.5941 - val_loss: 0.9293 - val_accuracy: 0.6288
Epoch 4/5
29/29 [==============================] - 0s 4ms/step - loss: 1.0109 - accuracy: 0.5941 - val_loss: 0.9138 - val_accuracy: 0.6157
Epoch 5/5
8/8 [==============================] - 0s 2ms/step - loss: 0.8904 - accuracy: 0.6594
Epoch 1/5
29/29 [==============================] - 2s 16ms/step - loss: 1.7971 - accuracy: 0.4234 - val_loss: 1.17

29/29 [==============================] - 1s 11ms/step - loss: 1.5366 - accuracy: 0.5044 - val_loss: 1.0247 - val_accuracy: 0.6026
Epoch 2/5
29/29 [==============================] - 0s 5ms/step - loss: 1.0508 - accuracy: 0.6149 - val_loss: 0.9382 - val_accuracy: 0.6288
Epoch 3/5
29/29 [==============================] - 0s 5ms/step - loss: 1.0007 - accuracy: 0.6039 - val_loss: 0.9508 - val_accuracy: 0.5764
Epoch 4/5
29/29 [==============================] - 0s 5ms/step - loss: 0.9539 - accuracy: 0.6171 - val_loss: 0.8966 - val_accuracy: 0.6332
Epoch 5/5
8/8 [==============================] - 0s 2ms/step - loss: 0.9063 - accuracy: 0.6114
Epoch 1/5
29/29 [==============================] - 1s 13ms/step - loss: 2.3734 - accuracy: 0.1641 - val_loss: 2.1461 - val_accuracy: 0.4279
Epoch 2/5
29/29 [==============================] - 0s 4ms/step - loss: 1.9933 - accuracy: 0.4606 - val_loss: 1.7905 - val_accuracy: 0.4978
Epoch 3/5
29/29 [==============================] - 0s 5ms/step - loss: 1.6983 -

...

8/8 [==============================] - 0s 3ms/step - loss: 0.9266 - accuracy: 0.6332
Epoch 1/20
29/29 [==============================] - 1s 13ms/step - loss: 1.3189 - accuracy: 0.5284 - val_loss: 0.9347 - val_accuracy: 0.6507
Epoch 2/20
29/29 [==============================] - 0s 5ms/step - loss: 1.0230 - accuracy: 0.6083 - val_loss: 0.8912 - val_accuracy: 0.6419
Epoch 3/20
29/29 [==============================] - 0s 5ms/step - loss: 0.9767 - accuracy: 0.6083 - val_loss: 0.8928 - val_accuracy: 0.6026
Epoch 4/20
29/29 [==============================] - 0s 5ms/step - loss: 0.9314 - accuracy: 0.6182 - val_loss: 0.8797 - val_accuracy: 0.6376
Epoch 5/20
29/29 [==============================] - 0s 5ms/step - loss: 0.9168 - accuracy: 0.6127 - val_loss: 0.8939 - val_accuracy: 0.6114
Epoch 6/20
29/29 [==============================] - 0s 6ms/step - loss: 0.8846 - accuracy: 0.6258 - val_loss: 0.8730 - val_accuracy: 0.6201
Epoch 7/20
29/29 [==============================] - 0s 5ms/step - loss: 0.

29/29 [==============================] - 0s 5ms/step - loss: 1.0225 - accuracy: 0.6094 - val_loss: 0.9317 - val_accuracy: 0.6157
Epoch 18/20
29/29 [==============================] - 0s 5ms/step - loss: 1.0122 - accuracy: 0.6028 - val_loss: 0.9255 - val_accuracy: 0.6550
Epoch 19/20
29/29 [==============================] - 0s 5ms/step - loss: 1.0048 - accuracy: 0.6116 - val_loss: 0.9223 - val_accuracy: 0.6463
Epoch 20/20
8/8 [==============================] - 0s 3ms/step - loss: 0.9166 - accuracy: 0.6594
Epoch 1/20
29/29 [==============================] - 1s 13ms/step - loss: 1.2801 - accuracy: 0.5295 - val_loss: 0.9593 - val_accuracy: 0.6157
Epoch 2/20
29/29 [==============================] - 0s 5ms/step - loss: 1.0142 - accuracy: 0.6007 - val_loss: 0.9438 - val_accuracy: 0.5939
Epoch 3/20
29/29 [==============================] - 0s 5ms/step - loss: 0.9793 - accuracy: 0.6039 - val_loss: 0.9007 - val_accuracy: 0.6114
Epoch 4/20
29/29 [==============================] - 0s 5ms/step - loss:

29/29 [==============================] - 0s 5ms/step - loss: 1.0654 - accuracy: 0.5919 - val_loss: 0.9743 - val_accuracy: 0.6114
Epoch 14/20
29/29 [==============================] - 0s 5ms/step - loss: 1.0628 - accuracy: 0.5853 - val_loss: 0.9643 - val_accuracy: 0.6114
Epoch 15/20
29/29 [==============================] - 0s 4ms/step - loss: 1.0490 - accuracy: 0.5952 - val_loss: 0.9564 - val_accuracy: 0.6070
Epoch 16/20
29/29 [==============================] - 0s 5ms/step - loss: 1.0372 - accuracy: 0.6018 - val_loss: 0.9506 - val_accuracy: 0.6157
Epoch 17/20
29/29 [==============================] - 0s 5ms/step - loss: 1.0347 - accuracy: 0.6007 - val_loss: 0.9452 - val_accuracy: 0.6070
Epoch 18/20
29/29 [==============================] - 0s 5ms/step - loss: 1.0221 - accuracy: 0.6072 - val_loss: 0.9380 - val_accuracy: 0.6201
Epoch 19/20
29/29 [==============================] - 0s 4ms/step - loss: 1.0279 - accuracy: 0.6050 - val_loss: 0.9366 - val_accuracy: 0.6026
Epoch 20/20
8/8 [========

29/29 [==============================] - 0s 4ms/step - loss: 1.1349 - accuracy: 0.5700 - val_loss: 1.0329 - val_accuracy: 0.6070
Epoch 10/20
29/29 [==============================] - 0s 5ms/step - loss: 1.1345 - accuracy: 0.5624 - val_loss: 1.0135 - val_accuracy: 0.6114
Epoch 11/20
29/29 [==============================] - 0s 4ms/step - loss: 1.1010 - accuracy: 0.5744 - val_loss: 0.9957 - val_accuracy: 0.6114
Epoch 12/20
29/29 [==============================] - 0s 4ms/step - loss: 1.0920 - accuracy: 0.5755 - val_loss: 0.9829 - val_accuracy: 0.6114
Epoch 13/20
29/29 [==============================] - 0s 4ms/step - loss: 1.0703 - accuracy: 0.5864 - val_loss: 0.9705 - val_accuracy: 0.6114
Epoch 14/20
29/29 [==============================] - 0s 5ms/step - loss: 1.0717 - accuracy: 0.5799 - val_loss: 0.9629 - val_accuracy: 0.6245
Epoch 15/20
29/29 [==============================] - 0s 4ms/step - loss: 1.0579 - accuracy: 0.5799 - val_loss: 0.9548 - val_accuracy: 0.6201
Epoch 16/20
29/29 [======

29/29 [==============================] - 0s 5ms/step - loss: 1.3610 - accuracy: 0.5470 - val_loss: 1.2433 - val_accuracy: 0.5939
Epoch 6/20
29/29 [==============================] - 0s 5ms/step - loss: 1.2961 - accuracy: 0.5142 - val_loss: 1.1754 - val_accuracy: 0.5852
Epoch 7/20
29/29 [==============================] - 0s 7ms/step - loss: 1.2455 - accuracy: 0.5470 - val_loss: 1.1271 - val_accuracy: 0.5939
Epoch 8/20
29/29 [==============================] - 0s 6ms/step - loss: 1.1935 - accuracy: 0.5569 - val_loss: 1.0902 - val_accuracy: 0.5895
Epoch 9/20
29/29 [==============================] - 0s 5ms/step - loss: 1.1749 - accuracy: 0.5438 - val_loss: 1.0624 - val_accuracy: 0.5939
Epoch 10/20
29/29 [==============================] - 0s 6ms/step - loss: 1.1310 - accuracy: 0.5821 - val_loss: 1.0403 - val_accuracy: 0.5983
Epoch 11/20
29/29 [==============================] - 0s 8ms/step - loss: 1.1218 - accuracy: 0.5722 - val_loss: 1.0234 - val_accuracy: 0.5983
Epoch 12/20
29/29 [==========

In [382]:
end_time = datetime.datetime.now()

In [383]:
run_time = end_time - start_time
run_time.total_seconds()
seconds_total = run_time.total_seconds()
hours   = divmod(seconds_total, 3600)
minutes = divmod(hours[1], 60)
seconds = divmod(minutes[1], 1)
print("Total runtime with itteration thru the parameters: %d hours, %d minutes and %d seconds" % (hours[0], minutes[0], seconds[0]))

Total runtime with itteration thru the parameters: 0 hours, 12 minutes and 5 seconds


In [384]:
test_results.sort_values(by='accuracy', ascending=False).head()

,epochs,neurons,dropout,Optimazion type,accuracy
32,5,128,0.4,rmsprop,0.659389
205,20,256,0.2,sgd,0.659389
153,15,256,0.3,adam,0.655022
122,15,64,0.4,rmsprop,0.655022
101,10,256,0.3,rmsprop,0.655022


In [385]:
test_results.sort_values(by='accuracy', ascending=False).tail()

,epochs,neurons,dropout,Optimazion type,accuracy
19,5,128,0.0,sgd,0.541485
25,5,128,0.2,sgd,0.541485
10,5,64,0.3,sgd,0.537118
28,5,128,0.3,sgd,0.537118
16,5,64,0.5,sgd,0.497817


In [386]:
test_results.to_csv('My_iterations/data_'+ start_time.strftime("%Y%m%d") +'_starttime' + start_time.strftime("%H%M") + '_duration' + str(hours[0]) + str(minutes[0]) + '.csv', index=False)  

### Delkonklusion ###  

#### Tensorflow Board ####  
  
  Tensorboardet gir et godt overblik over epochs og loss. Det bliver tydeligt hvornår, både epochs og loss begynder og flade ud og hvor der evt. kunne være brug for et early stop.  
  
Tensorboardet gir også et indtryk, i graferne for epochs vs. iteration og loss vs. iteration, af over og under fitting.  

#### Gennemløb ####
  
Ud fra tre gennemløb af vores test af modellen kan vi også i denne version, konkludere følgende med nogenlunde sikkerhed:  
  * Det betyder rigtig meget for modellens accuracy, hvordan den er sat sammen (vi får i dette tilfælde resultater fra ca. 49% til ca 66%).  
  * Vi kan efter de tre gennemløb se at vi overvejende får et godt resultat ved at bruge rmsprop optimeringsmetoden. Vi kan anvende sgd og adam, men det kræver et stort antal neuroner og mange epochs.  
  * Vi skal undgå at bruge sgd optimeringsmetoden sammen med lavt epochs og lavt neuron tal, da dette vil resultere i lav score.  
  
Vi svært ved at sige noget om:
  
  * Når vi anvender rmsprop som optimering, er det sværet at finde mønster i epochs og dropout.

Det vi kan sige er at det ser ud til at være mest fornuftigt at bruge rmsprop når man som i vores tilfælde bruger ANN og kun et lag. Det ser også, ikke uventet, ud til at et stort antal neuroner samt et rimeligt antal dropouts gir det bedste resultat.  
  

**Figur 1: Oversigt over tre bedste placeringer i tre gennemløb.**  
<img align="left" src="best.png"><br><br><br><br><br><br><br><br><br><br>
  
  
**Figur 2: Oversigt over tre værste placeringer i tre gennemløb.**  
<img align="left" src="worst.png"> 